In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input, Attention, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
import numpy as np

In [2]:
with open('data.txt', 'r', encoding='utf-8') as file:
        lines = file.readlines()
        word_list = [line.strip().lower() for line in lines]

# Sample data for outputs
vocab = word_list
characters = set(''.join(word_list))

# Encoding dictionaries
char_to_id = {char: i + 1 for i, char in enumerate(sorted(characters))}
char_to_id[" "] = 0
word_to_id = {word: i for i, word in enumerate(vocab)}

# Inverse for decoding
id_to_word = {i: word for word, i in word_to_id.items()}

max_char_length = max(len(word) for word in word_list)  # Maximum length of words in `data`

# Convert input words to encoded characters
X=[]
Y=[]
for word in word_list:
    for i in range(1,len(word)):
        x=[]
        for char in word[:i]:
            x.append(char_to_id[char])
        X.append(x)
        Y.append(word_to_id[word])

X_padded = pad_sequences(X, maxlen=max_char_length, padding='post')
X_padded = np.array(X_padded)
Y = np.array(Y)

In [3]:
# Hyperparameters
vocab_size = len(vocab)  # Number of words in the vocabulary
char_vocab_size = 256  # Assuming ASCII characters for simplicity
embedding_dim = 64
rnn_units = 128

# Model Definition
input_chars = Input(shape=(None,), dtype='int32')
char_embeddings = Embedding(input_dim=char_vocab_size, output_dim=embedding_dim)(input_chars)
rnn_out, state = SimpleRNN(units=rnn_units, return_sequences=True, return_state=True)(char_embeddings)

# Attention mechanism
query = tf.expand_dims(state, 1)  # Use the last RNN state as the query
attention_output, attention_weights = Attention(use_scale=True)([query, rnn_out], return_attention_scores=True)
flattened_output = Flatten()(attention_output)

# Output layer to predict the word index
word_pred = Dense(vocab_size, activation='softmax')(flattened_output)

model = Model(inputs=input_chars, outputs=word_pred)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 64)             16384     ['input_1[0][0]']             
                                                                                                  
 simple_rnn (SimpleRNN)      [(None, None, 128),          24704     ['embedding[0][0]']           
                              (None, 128)]                                                        
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 128)               0         ['simple_rnn[0][1]']    

In [4]:
model.fit(X_padded, Y, epochs=1000, batch_size=16)


Epoch 1/1000


3493/3493 [==============================] - 24s 7ms/step - loss: 8.6969 - accuracy: 0.0011
Epoch 2/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.3441 - accuracy: 0.0039
Epoch 3/1000
3493/3493 [==============================] - 24s 7ms/step - loss: 6.5132 - accuracy: 0.0110
Epoch 4/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 5.9494 - accuracy: 0.0201
Epoch 5/1000
3493/3493 [==============================] - 24s 7ms/step - loss: 5.5747 - accuracy: 0.0303
Epoch 6/1000
3493/3493 [==============================] - 24s 7ms/step - loss: 5.3442 - accuracy: 0.0395
Epoch 7/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 5.1662 - accuracy: 0.0480
Epoch 8/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 5.0522 - accuracy: 0.0563
Epoch 9/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 4.9327 - accuracy: 0.0639
Epoch 10/1000
3493/3493 [============================

3493/3493 [==============================] - 23s 7ms/step - loss: 7.5562 - accuracy: 0.0083
Epoch 75/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.5265 - accuracy: 0.0090
Epoch 76/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.4879 - accuracy: 0.0086
Epoch 77/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.7143 - accuracy: 0.0075
Epoch 78/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.7081 - accuracy: 0.0082
Epoch 79/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.8327 - accuracy: 0.0077
Epoch 80/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.8527 - accuracy: 0.0079
Epoch 81/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.7744 - accuracy: 0.0083
Epoch 82/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 7.6015 - accuracy: 0.0094
Epoch 83/1000
3493/3493 [==============================] - 2

3493/3493 [==============================] - 23s 7ms/step - loss: 3.2191 - accuracy: 0.2625
Epoch 227/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2283 - accuracy: 0.2624
Epoch 228/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2348 - accuracy: 0.2631
Epoch 229/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2262 - accuracy: 0.2618
Epoch 230/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2179 - accuracy: 0.2606
Epoch 231/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2205 - accuracy: 0.2618
Epoch 232/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2214 - accuracy: 0.2650
Epoch 233/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2252 - accuracy: 0.2635
Epoch 234/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2398 - accuracy: 0.2625
Epoch 235/1000
3493/3493 [==========================

3493/3493 [==============================] - 23s 7ms/step - loss: 3.8645 - accuracy: 0.2375
Epoch 379/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.8513 - accuracy: 0.2367
Epoch 380/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.7798 - accuracy: 0.2432
Epoch 381/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.9325 - accuracy: 0.2352
Epoch 382/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.8987 - accuracy: 0.2349
Epoch 383/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.8643 - accuracy: 0.2380
Epoch 384/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.8332 - accuracy: 0.2395
Epoch 385/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.7661 - accuracy: 0.2429
Epoch 386/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.6626 - accuracy: 0.2503
Epoch 387/1000
3493/3493 [==========================

3493/3493 [==============================] - 23s 7ms/step - loss: 3.7491 - accuracy: 0.2341
Epoch 531/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.6637 - accuracy: 0.2399
Epoch 532/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.6905 - accuracy: 0.2363
Epoch 533/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.7464 - accuracy: 0.2354
Epoch 534/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.7298 - accuracy: 0.2342
Epoch 535/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.7063 - accuracy: 0.2385
Epoch 536/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.6820 - accuracy: 0.2416
Epoch 537/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.8195 - accuracy: 0.2304
Epoch 538/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.9006 - accuracy: 0.2295
Epoch 539/1000
3493/3493 [==========================

3493/3493 [==============================] - 23s 7ms/step - loss: 3.2671 - accuracy: 0.2650
Epoch 683/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2670 - accuracy: 0.2631
Epoch 684/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2713 - accuracy: 0.2634
Epoch 685/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2728 - accuracy: 0.2644
Epoch 686/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2654 - accuracy: 0.2651
Epoch 687/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2749 - accuracy: 0.2632
Epoch 688/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2773 - accuracy: 0.2621
Epoch 689/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2658 - accuracy: 0.2631
Epoch 690/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2653 - accuracy: 0.2640
Epoch 691/1000
3493/3493 [==========================

3493/3493 [==============================] - 23s 7ms/step - loss: 3.3607 - accuracy: 0.2599
Epoch 835/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3519 - accuracy: 0.2607
Epoch 836/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3715 - accuracy: 0.2615
Epoch 837/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.4184 - accuracy: 0.2583
Epoch 838/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3659 - accuracy: 0.2599
Epoch 839/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3491 - accuracy: 0.2603
Epoch 840/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3323 - accuracy: 0.2595
Epoch 841/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3239 - accuracy: 0.2594
Epoch 842/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3102 - accuracy: 0.2629
Epoch 843/1000
3493/3493 [==========================

3493/3493 [==============================] - 23s 7ms/step - loss: 3.3775 - accuracy: 0.2594
Epoch 987/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3907 - accuracy: 0.2593
Epoch 988/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3043 - accuracy: 0.2598
Epoch 989/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3016 - accuracy: 0.2588
Epoch 990/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2934 - accuracy: 0.2618
Epoch 991/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2851 - accuracy: 0.2605
Epoch 992/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.2983 - accuracy: 0.2606
Epoch 993/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3028 - accuracy: 0.2599
Epoch 994/1000
3493/3493 [==============================] - 23s 7ms/step - loss: 3.3012 - accuracy: 0.2617
Epoch 995/1000
3493/3493 [==========================

In [5]:
model.save('03102024Final2_2.h5')
#model = load_model('03102024Final2.h5')

C:\Users\Stude\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
input_chars = "cat"  # Example input
input_ids = [char_to_id.get(char, 0) for char in input_chars]
input_padded = pad_sequences([input_ids], maxlen=max_char_length, padding='post')

# Make a prediction
prediction = model.predict(input_padded)
# Get the top 3 word IDs and their probabilities
top_3_indices = np.argsort(prediction[0])[-3:][::-1]  # Sort and get top 3 indices
top_3_probs = np.sort(prediction[0])[-3:][::-1]  # Sort and get top 3 probabilities
top_3_words = [id_to_word[idx] for idx in top_3_indices]
top_3_percentages = [f"{prob*100:.2f}%" for prob in top_3_probs]
for word, percentage in zip(top_3_words, top_3_percentages):
    print(f"Word: {word}, Softmax: {percentage}")

1/1 [==============================] - 0s 140ms/step
Word: catalyst, Softmax: 29.78%
Word: catalogue, Softmax: 14.21%
Word: catalog, Softmax: 9.85%
